In [1]:
import pandas as pd
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn import metrics

In [2]:
North_east_2 = pd.read_csv("North_east_2.csv")
North_east = pd.read_csv("North_east.csv")
North_west_2 = pd.read_csv("North_west_2.csv")
North_west_1 = pd.read_csv("North_west_1.csv")

In [3]:

dfs = [North_east_2, North_east,North_west_2,North_west_1]
df=pd.DataFrame()


for i in dfs:
    df = df.append(i)

data_mean=df.groupby('Datetime').mean()
data_mean.index = pd.to_datetime(data_mean.index)

In [4]:
data_mean.columns

Index(['air_temp_5cm_avg', 'air_temp_5cm_max', 'air_temp_5cm_min',
       'air_temp_avg', 'air_temp_max', 'air_temp_min', 'cloud_cover_avg',
       'cloud_cover_max', 'cloud_cover_min', 'dew_point_avg', 'dew_point_max',
       'dew_point_min', 'ice_acc_period', 'liquid_acc_period',
       'long_wave_radiation_avg', 'long_wave_radiation_max',
       'long_wave_radiation_min', 'pet_period', 'precip_acc_period',
       'precip_acc_period_adjusted', 'precip_acc_period_raw',
       'precip_duration', 'relative_humidity_avg', 'relative_humidity_max',
       'relative_humidity_min', 'short_wave_radiation_avg',
       'short_wave_radiation_max', 'short_wave_radiation_min',
       'snow_acc_period', 'sunshine_duration', 'visibility_avg',
       'visibility_max', 'visibility_min', 'wind_gust_max',
       'wind_speed_2m_avg', 'wind_speed_2m_max', 'wind_speed_2m_min',
       'wind_speed_avg', 'wind_speed_max', 'wind_speed_min'],
      dtype='object')

In [5]:
price =  pd.read_csv("London Wheat Futures Historical Data.csv")
price['Datetime'] = pd.to_datetime(price['Date'])
price = price.set_index(['Datetime'])
price = price.drop(['Date'], axis=1)
price =price.groupby(pd.Grouper(freq='10D'))['Price'].mean()

In [6]:

def rf_r2score(var,price):
    features = pd.get_dummies(var)
    labels = price
    features = np.array(features)
    labels = np.array(labels)

    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=100)

    rf_exp = RandomForestRegressor(n_estimators= 200, random_state=1000)
    rf_exp.fit(X_train, y_train)

    y_pred = rf_exp.predict(X_test)
    print("R2: ",r2_score(y_test,y_pred))
    
    errors = abs(y_pred - y_test)
    mape = 100 * (errors / y_test)
    accuracy = 100 - np.mean(mape)
    print("mape accuracy: ",accuracy)
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [7]:
var=pd.DataFrame()
data_copy = data_mean

data_copy['frost_days'] = data_mean[data_mean.air_temp_min<0 ]['air_temp_min']
data_copy['ice_days'] = data_mean[data_mean.air_temp_max<0 ]['air_temp_max']
data_copy['tem_10_25'] = data_mean[(10<data_mean.air_temp_avg)&(data_mean.air_temp_avg<25) ]['air_temp_avg']
data_copy['wet_days'] = data_mean[data_mean.precip_acc_period>1 ]['precip_acc_period']
data_copy['very_heavy_precip_days'] = data_mean[data_mean.precip_acc_period>20 ]['precip_acc_period']
data_copy['heavy_precip_days'] = data_mean[data_mean.precip_acc_period>10 ]['precip_acc_period']
data_copy['Summer_days'] = data_mean[data_mean.air_temp_max>25 ]['air_temp_max']
data_copy['precip_sum'] = data_mean['precip_acc_period']


data = data_copy.groupby(pd.Grouper(freq='10D')).agg({'frost_days':'count', 
                                               'ice_days':'count',
                                               'tem_10_25':'count',
                                               'very_heavy_precip_days':'count',
                                               'heavy_precip_days':'count',
                                               'Summer_days':'count',
                                               'wet_days':'count',
                                             'precip_sum':'sum', 
                                              'air_temp_max':['max','min'],'air_temp_min':['max','min'],'air_temp_avg':'mean'
                         })



In [8]:
winter =data.loc[(data.index.month.isin([11, 12, 1, 2, 3]))]
winter_price = price.loc[(price.index.month.isin([11, 12, 1, 2, 3]))]
winter.corrwith(winter_price)

frost_days              count    0.081265
ice_days                count    0.122242
tem_10_25               count   -0.086261
very_heavy_precip_days  count   -0.128711
heavy_precip_days       count   -0.014160
Summer_days             count         NaN
wet_days                count   -0.121729
precip_sum              sum     -0.090219
air_temp_max            max     -0.015702
                        min     -0.151064
air_temp_min            max     -0.003055
                        min     -0.063806
air_temp_avg            mean    -0.098737
dtype: float64

In [9]:
spring =data.loc[(data.index.month.isin([4,5]))]
spring_price = price.loc[(price.index.month.isin([4,5]))]
spring.corrwith(spring_price)

frost_days              count    0.143518
ice_days                count         NaN
tem_10_25               count    0.126257
very_heavy_precip_days  count   -0.006345
heavy_precip_days       count    0.076416
Summer_days             count   -0.057291
wet_days                count   -0.028974
precip_sum              sum      0.009096
air_temp_max            max      0.031794
                        min      0.155579
air_temp_min            max      0.101512
                        min     -0.001411
air_temp_avg            mean     0.077643
dtype: float64

In [10]:
summer =data.loc[(data.index.month.isin([6,7]))]
summer_price = price.loc[(price.index.month.isin([6,7]))]
summer.corrwith(summer_price)

frost_days              count         NaN
ice_days                count         NaN
tem_10_25               count    0.192681
very_heavy_precip_days  count    0.002853
heavy_precip_days       count    0.126702
Summer_days             count    0.110207
wet_days                count    0.202822
precip_sum              sum      0.213097
air_temp_max            max     -0.001895
                        min     -0.033332
air_temp_min            max     -0.113711
                        min     -0.059166
air_temp_avg            mean    -0.054537
dtype: float64

In [11]:
 rf_r2score(summer_price,summer)

R2:  0.11843938443140047
mape accuracy:  nan
Root Mean Squared Error: 4.006300369274146


<ipython-input-6-e19da6888bf4>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * (errors / y_test)
<ipython-input-6-e19da6888bf4>:16: RuntimeWarning: invalid value encountered in true_divide
  mape = 100 * (errors / y_test)


In [12]:
rf_r2score(spring_price,spring)

R2:  0.06488595771511929
mape accuracy:  nan
Root Mean Squared Error: 4.443377646765962


<ipython-input-6-e19da6888bf4>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * (errors / y_test)
<ipython-input-6-e19da6888bf4>:16: RuntimeWarning: invalid value encountered in true_divide
  mape = 100 * (errors / y_test)


In [13]:
rf_r2score(winter_price,winter)

R2:  0.046077006560352136
mape accuracy:  nan
Root Mean Squared Error: 4.399460236392398


<ipython-input-6-e19da6888bf4>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * (errors / y_test)
<ipython-input-6-e19da6888bf4>:16: RuntimeWarning: invalid value encountered in true_divide
  mape = 100 * (errors / y_test)
